<a href="https://colab.research.google.com/github/blackwithwhitegreen/Transformer/blob/main/Building_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-12 07:20:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-03-12 07:20:04 (17.0 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [21]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [22]:
print("length of dataset in char",len(text))

length of dataset in char 1115394


In [23]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [24]:
# here are all the unique char. that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [25]:
# Create a mapping from characters to integers.
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encode: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode : take a list of integer, output a string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [26]:
#let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.type)
print(data[:1000]) #the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) <built-in method type of Tensor object at 0x7f69338128d0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [27]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val/train
train_data = data[:n]
val_data = data[n:]

In [28]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [29]:
x = train_data[:block_size] # Inputs for the transformers.
y = train_data[1:block_size+1] # Next block size characters, y are are the targests for the each inputs
for t in range (block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [30]:
torch.manual_seed(1337) # For generating random numbers using manual_seed()
batch_size = 4 # How many independent sequences wil we process in parallel?
block_size = 8 # What is the maximum context length for predictions?

def get_batch(split): # split for any arbitory split
  # Generate a small batch of data of inputs x and targets y.
  # if the split is training split then it is train split otherwise val_data.
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # Batch Dimension.
  for t in range(block_size): # Time Dimension.
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53,

In [50]:
print(xb) # input to the transformer

tensor([[43, 43, 58, 43, 57, 58,  1, 46],
        [17, 30, 16, 21, 32, 13, 10,  0],
        [40, 43,  1, 39,  1, 56, 43, 39],
        [43, 57, 58,  1, 53, 44, 44, 47],
        [47, 52,  1, 58, 46, 39, 58,  1],
        [10,  0, 26, 39, 63,  6,  1, 21],
        [53, 59, 58, 46, 10,  0, 35, 46],
        [ 1, 51, 43, 57, 57, 43, 57,  0],
        [60, 43, 11,  0, 13, 52, 42,  1],
        [19,  1, 30, 21, 15, 20, 13, 30],
        [17, 32, 10,  0, 32, 46, 39, 58],
        [24, 21, 17, 32, 10,  0, 26, 53],
        [57,  8,  1, 21,  1, 51, 59, 57],
        [56, 43,  1, 57, 39, 60, 39, 45],
        [ 1, 50, 47, 60, 43,  6,  1, 54],
        [53, 52, 53, 59, 56,  1, 58, 56],
        [56, 57,  1, 45, 53, 53, 42,  1],
        [43, 52,  6,  1, 61, 47, 58, 46],
        [59, 56,  1, 51, 53, 58, 46, 43],
        [39, 52, 42,  6,  1, 47, 52,  1],
        [59, 52, 43, 57,  1, 39, 56, 43],
        [52, 53, 59, 57,  1, 47, 52,  1],
        [ 1, 42, 43, 61,  1, 53, 44,  1],
        [58, 46,  1, 17, 52, 45, 5

In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token form a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):

    #idx and targets are both(B,T) tensor of integers
    logits = self.token_embedding_table(idx) #(B,T,C)

    # We need to reshape out logits.
    if targets is None:
      loss =  None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    # idx is (B,T) array of indices in the current context.
    for _ in range (max_new_tokens):
      # get thr predictions
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:,-1,:] # because(B,C)
      # apply softmax to get probabilities
      probs = F.softmax(logits,dim=-1) # (B,C)
      # samples from the distribution
      idx_next = torch.multinomial(probs,num_samples=1) #(B,1)
      # append sampled index to the running sequece
      idx = torch.cat((idx,idx_next),dim=1)# (B,T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.6358, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [54]:
# Create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [55]:
batch_size = 32
for setps in range(10000):

  # sample a batch of data
  xb,yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.5727508068084717


#Here we can clearly see that the tokens are not talking to each other, so due to this it produces the bad results.

In [56]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=400)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenou


#The Mathematical trick in self-attention.

In [57]:
#Consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [58]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bow = bag of words
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t,c)
    xbow[b,t] = torch.mean(xprev,0)

In [62]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [61]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [63]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('__')
print('b=')
print(b)
print('__')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
__
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
__
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
